# image re-labelling code
going over the images i have already labelled, and re-labelling them.

In [1]:
import numpy as np
import pandas as pd
import os
#from PIL import Image
from glob import glob
import csv
import matplotlib.pyplot as plt
from IPython.display import clear_output
import math
import shutil

from numba import jit
import h5py

In [ ]:
number_of_interest = 11 # between 1 to 9, or 11

In [ ]:
# okay we are going to need a completly different approach for this
def v2_saving_function(df,save_path):
    df.to_csv(save_path, index=False)
    print(f'{saving_csv_name} saved!')
    return df

# make a list of index where these images are located
interesting_idx = csv_read_pd.index[csv_read_pd['number_label'] == number_of_interest].tolist()

for i in range(len(interesting_idx)):
#for i in range(10):
    idx = interesting_idx[i]
    row = csv_read_pd.loc[idx]
    image_name = row.iloc[0]
    og_label = row.iloc[1]
    new_label = row.iloc[2]

    # I've already labelled it a second time
    if not pd.isna(new_label):
        continue

    ## lets do the whole image labelling thing
    else:
        print(image_name)
        im = Image.open(os.path.join(folder_path, image_name + '.png'))
        plt.figure(figsize=(8, 8))
        plt.imshow(im)
        plt.axis('off')
        plt.show()
        while True: # loop will continue until a valid entry is passed
            inp = input("Enter a label (0-9 or 11, or 'stop' to quit): ") ## label of image

            ## if write 'stop' will stop labelling + save what has been done
            if inp.lower() == 'stop':
                print("Stopped labeling.")
                updated_df = v2_saving_function(csv_read_pd,full_save_csv_path)
                raise SystemExit  # Exit gracefully
            
            try:
                num = int(inp) # convert to number (corresponding to label)
                if 0 <= num <= 11:  # Check if the number is in the valid range
                    clear_output(wait=True) # get rid of image once you've labelled it
                    csv_read_pd.loc[idx, '2nd_label'] = num ## as working in a df, we over-write the correct cell
                    break  # Exit the loop if the input is valid
                else:
                    print("Error: Label must be between 0 and 11.")
            except ValueError: # if we don't input a number -> stop labelling, so will break
                print("not words, what are you doing???")
        
updated_df = v2_saving_function(csv_read_pd,full_save_csv_path)

## hide extra stuff

In [ ]:
csv_read_pd['2nd_label'].value_counts().get(1.0, 0)
#csv_read_pd['2nd_label'].count()
#df.to_csv(csv_file_list[0], index=False) 

In [ ]:
## copy newly labelled csv to github too
git_path = '/home/users/esree/CNN_basics/save_progress/'
shutil.copy(full_save_csv_path, git_path)

In [ ]:
og_csv = '2ds_10000.csv' ## original file
new_csv = '2ds_10000_8_dif.csv' ## new labels - in second column, but not all are second labelled
og_df = pd.read_csv(folder_path+og_csv)
new_df = pd.read_csv(folder_path+new_csv)

In [ ]:
## combining og labels + new labels - replace old labels
length_of_data = len(og_df['image_name'])
name_lst = []
label_lst = []

for idx in range(int(length_of_data)):
    row = csv_read_pd.loc[idx]
    image_name = row.iloc[0]
    og_label = row.iloc[1]
    new_label = row.iloc[2]
    
    name_lst.append(image_name)
    if not pd.isna(new_label):
        label_lst.append(new_label)
    else:
        label_lst.append(og_label)

df_dic = {'image_name':name_lst,'number_label':label_lst}
new_df = pd.DataFrame(df_dic)
new_df

In [ ]:
new_df['number_label'].count()
saving_csv_name = '2ds_10000_new.csv'
#saved_df = v2_saving_function(new_df,folder_path+saving_csv_name)

# group images together to download them

In [ ]:
folder_path = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/'
csv_file_name = '2ds_10000_new.csv'
csv_read_pd = pd.read_csv(folder_path+csv_file_name)

category_dic = {'1.0':'CP','2.0':'FA','3.0':'Co','4.0':'HPC','6.0':'CBC','7.0':'CA','8.0':'CC','9.0':'WD','0.0':'DIF','11.0':'unknown'}
category_dic_extra = {} # adding in path to loc 
for key in category_dic:
    var_name = category_dic[key]
    individ_cat_path = folder_path+var_name+'/'
    category_dic_extra[key] = [var_name,individ_cat_path]

'''
## make bunch of directories with appropriate names
category_names_lst = [category_dic[key] for key in category_dic] ## ayy make a list of category names from our dictionary values # newlist = [expression for item in iterable if condition == True]

# make each name a folder
for cat_name in category_names_lst:
    path = folder_path+cat_name+'/'
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"{cat_name} created successfully!")
    else:
        print(f"{cat_name} already exists.")
'''


length_of_data = len(csv_read_pd['image_name'])
for idx in range(int(length_of_data/1000)):
    row = csv_read_pd.loc[idx]
    image_name = row.iloc[0]
    label = row.iloc[1]
    og_loc = folder_path+image_name+'.png'
    new_loc = category_dic_extra[str(label)][1]
    
    shutil.copy(og_loc, new_loc)

print('copying images finished')

# zip up the folders with images in
for key in category_dic_extra:
    cat_folder = category_dic_extra[key][0]
    folder_path_to_zip = category_dic_extra[key][1][:-1]
    shutil.make_archive(folder_path_to_zip, 'zip',folder_path_to_zip) # as we are saving with same name but with .zip
    print(f'{cat_folder} zipped!')

In [ ]:
category_names

In [ ]:
csv_read_pd[:10]

# extract relevant particle stats of ones labelled

In [ ]:
path_2ds_stats = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/2ds/all_2ds.csv'
path_labelled = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2ds_10000_new_2nd.csv'

full_ds_stats = pd.read_csv(path_2ds_stats)
image_labelled_df = pd.read_csv(path_labelled)

images_labelled_lst = list(image_labelled_df['image_name'])

labelled_image_stats = full_ds_stats[full_ds_stats['name'].isin(images_labelled_lst)]


## need to add new column - with the label on
# not sure why 2 extra rows on df more than have been labelled??

## i have now discoved i have duplicate data, but i will have only extracted one of the images. but i don't know which one

In [ ]:
duplicates_mask = labelled_image_stats.duplicated(subset=['name'])
duplicates_df = labelled_image_stats[duplicates_mask]
duplicates_df

In [ ]:
duplicate_1 = labelled_image_stats[labelled_image_stats['name'] == '2484896_25ch1']
duplicate_2 = labelled_image_stats[labelled_image_stats['name'] == '47102_07ch1']

duplicate_2

In [ ]:
new_csv = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2ds_10000_new_2nd.csv'
full_ds_stats = pd.read_csv(new_csv)
 

full_ds_stats['note'].count()

In [ ]:


full_duplicates_mask = full_ds_stats.duplicated(subset=['name'])
full_duplicates_df = full_ds_stats[full_duplicates_mask]
full_duplicates_df

list_of_duplicates = list(full_duplicates_df['name'])
double_duplicates_df = full_ds_stats[full_ds_stats['name'].isin(list_of_duplicates)]

double_duplicates_df

In [ ]:
length_of_data = len(full_ds_stats['image_name'])

for idx in range(length_of_data):
    row = full_ds_stats.loc[idx]
    image_name = row.iloc[0]
    label = row.iloc[1]
    note = row.iloc[2]
    #og_loc = folder_path+image_name+'.png'
    #new_loc = category_dic_extra[str(label)][1]
    if not note != 'a':
        print('yep')
	    #shutil.copy(og_loc, new_loc)

In [ ]:
full_duplicates_df

#double_duplicates_df.to_csv('duplicates_df.csv', index=False)
reduced_duplicates = full_duplicates_df[:10]
reduced_duplicates.to_csv('/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/red_duplicates_df.csv', index=False)

## fixing that because i am stupid

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from glob import glob
import seaborn as sns
import pandas as pd
from scipy.special import gamma
from datetime import datetime, timedelta

import h5py ####
from PIL import Image
import os
import psutil

from scipy.ndimage import convolve, label
from skimage.measure import regionprops, find_contours
from scipy.spatial import ConvexHull, distance_matrix
from scipy.spatial.distance import pdist
from skimage.morphology import remove_small_holes ## remove holes <3
from scipy.ndimage import binary_fill_holes
from skimage import measure
import tensorflow as tf

from numba import jit

2025-01-28 12:01:29.477169: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
## og functions / things to make script run smoothly
## setting thresholds / res for attaining good particle final images
fill_hole_threshold = 5 # max number pixels contained within particle that is filled in

minimum_area = 20 # very quick metric to stop the processing of particles with area < 15 pixels
desired_image_size = 200 
pixel_resolution = 10 # mu for 2DS // need to be 150 for hvps
length_threshold = 100 ## 100 mu for 2ds // 1500 mu for hvps

def stats_description(bw_crystal, fill_hole_thresh):
    #take binary image, fill in small holes and returns object containing stats about crystal
    
    filled_particle = remove_small_holes(bw_crystal.image, area_threshold=fill_hole_thresh) # fill in voids within binary image - better estimation of stats # may need to be altered
       
    if filled_particle.shape[0] < 2 or filled_particle.shape[1] < 2:
        return filled_particle, None
        
    contours = measure.find_contours(filled_particle, 0.5)
    if contours:
        contour = max(contours, key=lambda x: x.shape[0])  # Sort contours by area (largest first) and select the largest contour
        
        labeled_image = measure.label(filled_particle)  # Label the image based on the threshold
        region = measure.regionprops(labeled_image)[0]  # Assumes largest labeled region corresponds to largest contour
        
        return filled_particle, region
    else:
        return filled_particle, None


## function to calculate truncation of particle
@jit(nopython=True) # Enables full optimization by numba
def calc_truncation(particle_coords):
    ## so much simpler, looking at list of coordinates making up a particle, then summing ones in 0 and 127 row - i.e. first + last diode
    lst_first_diode = [coord for coord in particle_coords if coord[0] == 0]
    lst_last_diode = [coord for coord in particle_coords if coord[0] == 127]

    n_top = len(lst_first_diode)
    n_bottom = len(lst_last_diode)

    return n_top, n_bottom # number pixels touching top / bottom respectively
## 

###  set up dataframe, used to extract from raw h5 file + has stats about the particle
columns = [
    "name",
    "date",
    "slice_s_idx",
    "slice_e_idx",
    #"start_time", #hh:mm:ss 
    #"end_time", #hh:mm:ss 
    "ellipse_d_max", # um
    #"ellipse_d_min", # um
    "Euclidean_d_max", # um
    "Feret_d_max", # um
    "area", # um2
    "perimeter", # um
    "circularity",
    "probe",
    "first_diode_trunc",
    "last_diode_trunc",
    "image_trunc",
    "aspect_ratio"
    ]


In [ ]:
# hopefully 
dup_csv_pth = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/duplicates_df.csv'
dup_df = pd.read_csv(dup_csv_pth)

#dup_df.sort_values(by=['name'])
#dup_df

duplicates_mask = dup_df.duplicated(subset=['name'])
one_dup_df = dup_df[duplicates_mask]
dup_name_lst = list(one_dup_df['name']) # list of 1 name of all the duplicates
row = dup_df[dup_df['name'] == dup_name_lst[6]] ## call out the specif rows which correspond to that name
#row_idx = list(row.index) ## can get specif index of rows selected
#for particle in row:
#    print(particle)
#row['name']
var = 'probe'
print(row[var])
print(max(row[var].values)) ## this is how we can call out the max values 

In [ ]:
for i in range(5):
    row = dup_df[dup_df['name'] == dup_name_lst[i]] # all rows with same name
    image_name = max(row['name'].values)
    date = str(max(row['date'].values))
    start_idx = max(row['slice_s_idx'].values)
    end_idx = max(row['slice_e_idx'].values)
    probe = max(row['probe'].values)
    date_day = max(row[var].values)
    date_day = date[-2:]

    #print(row)
    print(date_day)
    print(f'{start_idx} {end_idx}')

In [ ]:
## first attempt at fixing - just looping through rows wasnt working
#dup_csv_pth = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/red_duplicates_df.csv'
dup_csv_pth = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/duplicates_df.csv'
save_loc = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/'

base_h5_pth = '/gws/nopw/j04/dcmex/users/ezriab/raw_h5/2ds/'

particle_df = pd.DataFrame(columns=columns)
with open(dup_csv_pth, mode='r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)  # Read the header row
    # loop through each row
    for row in csv_reader:
        image_name = row[0]
        date = row[1] 
        start_idx = row[2]
        end_idx = row[3]
        probe = row[12]
        date_day = date[-2:]
        
        h5_of_interest = [] ## some have multiple files associated with each date
        list_h5_files = glob(base_h5_pth+probe+'/'+'*.h5')
        for file in list_h5_files:
            if date in file:
                h5_of_interest.append(file)

        for h5_path in h5_of_interest:
            h5_file = h5py.File(h5_path,'r')

            try:
                h5_image = h5_file['ImageData']
                h5_time = h5_file['ImageTimes']
            except KeyError as e:
                print(f"Dataset missing in file: {file}. Error: {e}")
                continue

            one_crystal = h5_file['ImageData'][:,int(start_idx)-2:int(end_idx)+2] # extract 1 crystal
            #plt.imshow(one_crystal, cmap='gray')
            binary_image = (one_crystal == 0) 
            labeled_image, num_features = label(binary_image)
            try:
                props = regionprops(labeled_image)
            except ValueError as e:
                print('something has gone wrong')
                continue
                
            for particle in props:
                if particle.area >= minimum_area:
                    filled_part, spec_region = stats_description(particle,fill_hole_threshold)
                    if spec_region:
                        aspect_ratio_value = spec_region.major_axis_length / spec_region.minor_axis_length
                        coords = particle.coords # basically gives coords of each point of interest [row,column]
                        distances = pdist(coords)
                        euclidean_dim = np.max(distances)

                        if euclidean_dim * pixel_resolution >= length_threshold:
                                
                            ## basic info
                            x_values = np.unique(coords[:, 1])
                            #s_idx = int(selected_pix_sum[i] + x_values[0])
                            #e_idx = int(selected_pix_sum[i] + x_values[-1])
            
                            ## truncation calc
                            # cnn - in final pic size, particle may be truncated if very long, this tells us how many pixels may be cut off 
                            image_trunc = x_values[-1] - desired_image_size 
                            if image_trunc < 0:
                                image_trunc = 0
                            # normal trunc calculation - on actual probe
                            first_diode, last_diode = calc_truncation(coords)
                            
                            ## using circularity calculation from Crosier et al. 2011
                            circularity_calc = np.divide((spec_region.perimeter**2),(4*np.pi*spec_region.area))
                            particle_name = f'{start_idx}_{particle.label}_{date_day}{probe}'
                            
                            # nice way of saving data - lenth + measurements are correct in microns
                            one_particle_data = {
                                    #"image_index": image_index,
                                    "name": particle_name,
                                    "date" : date,
                                    "slice_s_idx": start_idx,
                                    "slice_e_idx": end_idx,
                                    #"start_time": str(selected_utc_time[i].values).split('T')[1], # more friendly time
                                    #"end_time": str(selected_utc_time[i+1].values).split('T')[1], # more friendly time
                                    "ellipse_d_max": spec_region.major_axis_length * pixel_resolution, ## d_max (equivalent ellipse)
                                    #"ellipse_d_min": spec_region.minor_axis_length * pixel_resolution, ## d_min (equivalent ellipse)
                                    "Euclidean_d_max": euclidean_dim * pixel_resolution,
                                    "Feret_d_max":spec_region.feret_diameter_max * pixel_resolution,
                                    "area": (spec_region.area * (pixel_resolution**2)),
                                    "perimeter": (spec_region.perimeter * pixel_resolution),
                                    "circularity": circularity_calc,
                                    "probe": probe,
                                    "first_diode_trunc": first_diode,
                                    "last_diode_trunc": last_diode,
                                    "image_trunc": image_trunc,
                                    "aspect_ratio": aspect_ratio_value  
                                    }

                            filled_part = filled_part.astype(np.float32) ## convert to float 0 and 1s
                            filled_part = np.expand_dims(filled_part, axis=-1) ## add extra dimention - this is for adding padding
            
                            imagex = tf.image.resize_with_crop_or_pad(filled_part, desired_image_size, desired_image_size)
                            ## save image
                            # Remove the extra dimension if needed
                            image_np = imagex.numpy().squeeze()
                            #plt.imshow(image_np, cmap='gray')
                            plt.imsave(f'{save_loc}{particle_name}.png', image_np, cmap="gray")
                            

            h5_file.close()
            #print(one_particle_data)
            one_particle_data_df = pd.DataFrame([one_particle_data])
            particle_df = pd.concat([particle_df, one_particle_data_df], ignore_index=True)

particle_df.to_csv(f'{save_loc}updated_duplicates.csv', index=False)        
        

In [12]:
## second go - locate name pairs, then use the max end index 
#dup_csv_pth = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/red_duplicates_df.csv'
dup_csv_pth = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/duplicates_df.csv'
save_loc = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/do_it_again/'

base_h5_pth = '/gws/nopw/j04/dcmex/users/ezriab/raw_h5/2ds/'
particle_df = pd.DataFrame(columns=columns)

######## NEW ###########
dup_df = pd.read_csv(dup_csv_pth)
duplicates_mask = dup_df.duplicated(subset=['name'])
one_dup_df = dup_df[duplicates_mask]
dup_name_lst = list(one_dup_df['name']) # list of 1 name of all the duplicates
######################

'''
with open(dup_csv_pth, mode='r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)  # Read the header row
    # loop through each row
    for row in csv_reader:
        image_name = row[0]
        date = row[1] 
        start_idx = row[2]
        end_idx = row[3]
        probe = row[12]
        date_day = date[-2:]
'''
    ######## NEW ###########
for i in range(len(dup_name_lst)):
    row = dup_df[dup_df['name'] == dup_name_lst[i]] # all rows with same name
    image_name = max(row['name'].values)
    date = str(max(row['date'].values))
    start_idx = max(row['slice_s_idx'].values)
    end_idx = max(row['slice_e_idx'].values)
    probe = max(row['probe'].values)
    date_day = date[-2:]

    ########################
    print(image_name)    
    h5_of_interest = [] ## some have multiple files associated with each date
    list_h5_files = glob(base_h5_pth+probe+'/'+'*.h5')
    for file in list_h5_files:
        if date in file:
            h5_of_interest.append(file)

    for h5_path in h5_of_interest:
        h5_file = h5py.File(h5_path,'r')

        try:
            h5_image = h5_file['ImageData']
            h5_time = h5_file['ImageTimes']
        except KeyError as e:
            print(f"Dataset missing in file: {file}. Error: {e}")
            continue

        one_crystal = h5_file['ImageData'][:,int(start_idx):int(end_idx)] # extract 1 crystal
        #plt.imshow(one_crystal, cmap='gray')
        binary_image = (one_crystal == 0) 
        labeled_image, num_features = label(binary_image)
        try:
            props = regionprops(labeled_image)
        except ValueError as e:
            print(f' {file} at start idx {start_idx}, something has gone wrong')
            continue
            
        for particle in props:
            if particle.area >= minimum_area:
                filled_part, spec_region = stats_description(particle,fill_hole_threshold)
                if spec_region:
                    aspect_ratio_value = spec_region.major_axis_length / spec_region.minor_axis_length
                    coords = particle.coords # basically gives coords of each point of interest [row,column]
                    distances = pdist(coords)
                    euclidean_dim = np.max(distances)

                    if euclidean_dim * pixel_resolution >= length_threshold:
                            
                        ## basic info
                        x_values = np.unique(coords[:, 1])
                        e_idx = start_idx+x_values[-1]
                        #s_idx = int(selected_pix_sum[i] + x_values[0])
                        #e_idx = int(selected_pix_sum[i] + x_values[-1])
        
                        ## truncation calc
                        # cnn - in final pic size, particle may be truncated if very long, this tells us how many pixels may be cut off 
                        image_trunc = x_values[-1] - desired_image_size 
                        if image_trunc < 0:
                            image_trunc = 0
                        # normal trunc calculation - on actual probe
                        first_diode, last_diode = calc_truncation(coords)
                        
                        ## using circularity calculation from Crosier et al. 2011
                        circularity_calc = np.divide((spec_region.perimeter**2),(4*np.pi*spec_region.area))
                        particle_name = f'{start_idx}_{particle.label}_{date_day}{probe}'
                        
                        # nice way of saving data - lenth + measurements are correct in microns
                        one_particle_data = {
                                #"image_index": image_index,
                                "name": particle_name,
                                "date" : date,
                                "slice_s_idx": start_idx,
                                "slice_e_idx": e_idx,
                                #"start_time": str(selected_utc_time[i].values).split('T')[1], # more friendly time
                                #"end_time": str(selected_utc_time[i+1].values).split('T')[1], # more friendly time
                                "ellipse_d_max": spec_region.major_axis_length * pixel_resolution, ## d_max (equivalent ellipse)
                                #"ellipse_d_min": spec_region.minor_axis_length * pixel_resolution, ## d_min (equivalent ellipse)
                                "Euclidean_d_max": euclidean_dim * pixel_resolution,
                                "Feret_d_max":spec_region.feret_diameter_max * pixel_resolution,
                                "area": (spec_region.area * (pixel_resolution**2)),
                                "perimeter": (spec_region.perimeter * pixel_resolution),
                                "circularity": circularity_calc,
                                "probe": probe,
                                "first_diode_trunc": first_diode,
                                "last_diode_trunc": last_diode,
                                "image_trunc": image_trunc,
                                "aspect_ratio": aspect_ratio_value  
                                }

                        filled_part = filled_part.astype(np.float32) ## convert to float 0 and 1s
                        filled_part = np.expand_dims(filled_part, axis=-1) ## add extra dimention - this is for adding padding
        
                        imagex = tf.image.resize_with_crop_or_pad(filled_part, desired_image_size, desired_image_size)
                        ## save image
                        # Remove the extra dimension if needed
                        image_np = imagex.numpy().squeeze()
                        #plt.imshow(image_np, cmap='gray')
                        plt.imsave(f'{save_loc}{particle_name}.png', image_np, cmap="gray")
                        print(f'{particle_name} done')

                        one_particle_data_df = pd.DataFrame([one_particle_data])
                        
                        particle_df = pd.concat([particle_df, one_particle_data_df], ignore_index=True)
        print(f'{image_name} done')
        h5_file.close()
        #print(one_particle_data)

particle_df.to_csv(f'{save_loc}updated_duplicates_3nd.csv', index=False)        
        

94298_19ch0
94298_1_19ch0 done
94298_2_19ch0 done
94298_19ch0 done
226470_19ch0


/tmp/ipykernel_7116/2537520632.py:132: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  particle_df = pd.concat([particle_df, one_particle_data_df], ignore_index=True)


226470_1_19ch0 done
226470_3_19ch0 done
226470_19ch0 done
230869_19ch0
230869_1_19ch0 done
230869_4_19ch0 done
230869_19ch0 done
235921_19ch0
235921_1_19ch0 done
235921_3_19ch0 done
235921_19ch0 done
249985_19ch0
249985_1_19ch0 done
249985_2_19ch0 done
249985_19ch0 done
364115_19ch0
364115_1_19ch0 done
364115_13_19ch0 done
364115_19ch0 done
806361_19ch0
806361_1_19ch0 done
806361_2_19ch0 done
806361_19ch0 done
830185_19ch0
830185_1_19ch0 done
830185_2_19ch0 done
830185_19ch0 done
1188953_19ch0
1188953_1_19ch0 done
1188953_2_19ch0 done
1188953_19ch0 done
1403973_19ch0
1403973_1_19ch0 done
1403973_2_19ch0 done
1403973_19ch0 done
14000_20ch0
14000_1_20ch0 done
14000_5_20ch0 done
14000_20ch0 done
367051_20ch0
367051_1_20ch0 done
367051_3_20ch0 done
367051_20ch0 done
568208_20ch0
568208_1_20ch0 done
568208_3_20ch0 done
568208_20ch0 done
728277_20ch0
728277_1_20ch0 done
728277_2_20ch0 done
728277_20ch0 done
1003345_20ch0
1003345_1_20ch0 done
1003345_2_20ch0 done
1003345_20ch0 done
1016899_20

In [ ]:
particle_df

In [ ]:
re_dup_csv_pth = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/updated_duplicates_1st.csv'
dup_df_2 = pd.read_csv(re_dup_csv_pth)

full_duplicates_mask = dup_df_2.duplicated(subset=['name'])
full_duplicates_df = dup_df_2[full_duplicates_mask]
'''
list_of_duplicates = list(full_duplicates_df['name'])
double_duplicates_df = full_ds_stats[full_ds_stats['name'].isin(list_of_duplicates)]

double_duplicates_df
'''

full_duplicates_df


# test what have i done

In [ ]:
save_loc = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/'
csv_name = 'duplicates_df.csv'

dup_df = pd.read_csv(save_loc+csv_name)
dup_df.to_csv('full_dup.csv', index=False)  
#dup_df.to_csv()

In [ ]:
dup_df_1 = pd.read_csv(save_loc+'updated_duplicates_1st.csv')
duplicates_mask_1 = dup_df_1.duplicated(subset=['name'], keep ='last')
one_dup_df_1 = dup_df_1[duplicates_mask_1]
dup_df_1

In [ ]:
dup_df_2 = pd.read_csv(save_loc+'updated_duplicates_2nd.csv')
duplicates_mask_2 = dup_df_2.duplicated(subset=['name'])
one_dup_df_2 = dup_df_2[duplicates_mask_2]
one_dup_df_2

In [ ]:
dup_df_og = pd.read_csv(save_loc+'duplicates_df.csv')
duplicates_mask = dup_df_og.duplicated(subset=['name'])
one_dup_df_2 = dup_df_1[duplicates_mask]
one_dup_df_2

In [18]:
save_loc = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/do_it_again/'
csv_name = 'updated_duplicates_3nd.csv'

dup_df_3 = pd.read_csv(save_loc+csv_name)
duplicates_mask = dup_df_3.duplicated(subset=['name'],keep=False)
one_dup_df_3 = dup_df_3[duplicates_mask]
#dup_df_3.to_csv('dup_proc_3.csv', index=False)


In [15]:
duplicates_mask = one_dup_df_3.duplicated(subset=['name'])
another_filter = one_dup_df_3[duplicates_mask]
another_filter

,name,date,slice_s_idx,slice_e_idx,ellipse_d_max,Euclidean_d_max,Feret_d_max,area,perimeter,circularity,probe,first_diode_trunc,last_diode_trunc,image_trunc,aspect_ratio


In [2]:
path_2ds_stats = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/2ds/all_2ds.csv'
all_2ds = pd.read_csv(path_2ds_stats)
duplicates_mask = all_2ds.duplicated(subset=['name'], keep=False)
all_2ds_duplicates = all_2ds[duplicates_mask]
all_2ds_duplicates

,name,date,slice_s_idx,slice_e_idx,start_time,end_time,ellipse_d_max,Euclidean_d_max,Feret_d_max,area,...,first_diode_trunc,last_diode_trunc,image_trunc,aspect_ratio,wind_w,pres,alt,temp,LWC,IWC
3327,94298_19ch0,220719,94298,94332,18:53:04.977000000,18:53:04.977000000,349.710133,392.045916,402.243707,71300.0,...,25,0,0,1.219161,-2.043195,400.87717,7644.9404,257.90890,0.032683,0.076399
3328,94298_19ch0,220719,94298,94311,18:53:04.977000000,18:53:04.977000000,146.839578,130.000000,140.356688,4400.0,...,0,0,0,3.089049,-2.043195,400.87717,7644.9404,257.90890,0.032683,0.076399
8430,226470_19ch0,220719,226470,226482,18:56:50.694000000,18:56:50.694000000,152.272582,126.491106,143.178211,4200.0,...,0,0,0,3.395385,-2.762317,388.69946,7885.3310,255.70236,0.162223,0.334879
8431,226470_19ch0,220719,226470,226483,18:56:50.694000000,18:56:50.694000000,174.590712,172.046505,186.010752,13000.0,...,0,0,0,1.533949,-2.762317,388.69946,7885.3310,255.70236,0.162223,0.334879
8595,230869_19ch0,220719,230869,230879,18:56:50.772000000,18:56:50.772000000,135.556367,144.222051,152.643375,8000.0,...,0,0,0,1.263412,-2.762317,388.69946,7885.3310,255.70236,0.186962,0.442953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932424,315316_07ch1,220807,315316,315356,17:34:41.285000000,17:34:41.300000000,417.369190,512.249939,520.096145,88900.0,...,0,33,0,1.315120,-1.812491,416.33398,7310.0000,258.07930,0.019274,0.000000
939967,18539_07ch1,220807,18539,18595,19:18:02.117000000,19:18:02.117000000,579.097229,676.239603,688.186021,146400.0,...,6,0,0,1.589710,0.090431,431.38864,7046.8150,260.83000,0.000063,0.000000
940010,34578_07ch1,220807,34578,34691,19:20:07.116000000,19:20:40.110000000,977.017933,1136.353818,1146.298390,223500.0,...,0,83,0,3.120222,-3.798381,417.02188,7294.1480,259.73830,0.000284,0.000012
940011,34578_07ch1,220807,34578,34603,19:20:07.116000000,19:20:40.110000000,258.089121,259.615100,269.258240,8200.0,...,0,17,0,4.987685,-3.798381,417.02188,7294.1480,259.73830,0.000284,0.000012


In [28]:
#all_2ds_duplicates.to_csv('duplicates_df_v2.csv', index = False)

In [4]:
#duplicates_mask_2 = all_2ds.duplicated(subset=['name'], keep=True)
#all_2ds_duplicates_2 = all_2ds[duplicates_mask_2]
#all_2ds_duplicates_2
list_of_duplicates = list(all_2ds_duplicates['name'])
double_duplicates_df = all_2ds[all_2ds['name'].isin(list_of_duplicates)]
double_duplicates_df

,name,date,slice_s_idx,slice_e_idx,start_time,end_time,ellipse_d_max,Euclidean_d_max,Feret_d_max,area,...,first_diode_trunc,last_diode_trunc,image_trunc,aspect_ratio,wind_w,pres,alt,temp,LWC,IWC
3327,94298_19ch0,220719,94298,94332,18:53:04.977000000,18:53:04.977000000,349.710133,392.045916,402.243707,71300.0,...,25,0,0,1.219161,-2.043195,400.87717,7644.9404,257.90890,0.032683,0.076399
3328,94298_19ch0,220719,94298,94311,18:53:04.977000000,18:53:04.977000000,146.839578,130.000000,140.356688,4400.0,...,0,0,0,3.089049,-2.043195,400.87717,7644.9404,257.90890,0.032683,0.076399
8430,226470_19ch0,220719,226470,226482,18:56:50.694000000,18:56:50.694000000,152.272582,126.491106,143.178211,4200.0,...,0,0,0,3.395385,-2.762317,388.69946,7885.3310,255.70236,0.162223,0.334879
8431,226470_19ch0,220719,226470,226483,18:56:50.694000000,18:56:50.694000000,174.590712,172.046505,186.010752,13000.0,...,0,0,0,1.533949,-2.762317,388.69946,7885.3310,255.70236,0.162223,0.334879
8595,230869_19ch0,220719,230869,230879,18:56:50.772000000,18:56:50.772000000,135.556367,144.222051,152.643375,8000.0,...,0,0,0,1.263412,-2.762317,388.69946,7885.3310,255.70236,0.186962,0.442953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932424,315316_07ch1,220807,315316,315356,17:34:41.285000000,17:34:41.300000000,417.369190,512.249939,520.096145,88900.0,...,0,33,0,1.315120,-1.812491,416.33398,7310.0000,258.07930,0.019274,0.000000
939967,18539_07ch1,220807,18539,18595,19:18:02.117000000,19:18:02.117000000,579.097229,676.239603,688.186021,146400.0,...,6,0,0,1.589710,0.090431,431.38864,7046.8150,260.83000,0.000063,0.000000
940010,34578_07ch1,220807,34578,34691,19:20:07.116000000,19:20:40.110000000,977.017933,1136.353818,1146.298390,223500.0,...,0,83,0,3.120222,-3.798381,417.02188,7294.1480,259.73830,0.000284,0.000012
940011,34578_07ch1,220807,34578,34603,19:20:07.116000000,19:20:40.110000000,258.089121,259.615100,269.258240,8200.0,...,0,17,0,4.987685,-3.798381,417.02188,7294.1480,259.73830,0.000284,0.000012
